# Skip-gram word2vec

In this notebook, I'll lead you through using TensorFlow to implement the word2vec algorithm using the skip-gram architecture. By implementing this, you'll learn about embedding words for use in natural language processing. This will come in handy when dealing with things like machine translation.

## Readings

Here are the resources I used to build this notebook. I suggest reading these either beforehand or while you're working on this material.

* A really good [conceptual overview](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/) of word2vec from Chris McCormick 
* [First word2vec paper](https://arxiv.org/pdf/1301.3781.pdf) from Mikolov et al.
* [NIPS paper](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) with improvements for word2vec also from Mikolov et al.
* An [implementation of word2vec](http://www.thushv.com/natural_language_processing/word2vec-part-1-nlp-with-deep-learning-with-tensorflow-skip-gram/) from Thushan Ganegedara
* TensorFlow [word2vec tutorial](https://www.tensorflow.org/tutorials/word2vec)

## Word embeddings

When you're dealing with words in text, you end up with tens of thousands of classes to predict, one for each word. Trying to one-hot encode these words is massively inefficient, you'll have one element set to 1 and the other 50,000 set to 0. The matrix multiplication going into the first hidden layer will have almost all of the resulting values be zero. This a huge waste of computation. 

![one-hot encodings](assets/one_hot_encoding.png)

To solve this problem and greatly increase the efficiency of our networks, we use what are called embeddings. Embeddings are just a fully connected layer like you've seen before. We call this layer the embedding layer and the weights are embedding weights. We skip the multiplication into the embedding layer by instead directly grabbing the hidden layer values from the weight matrix. We can do this because the multiplication of a one-hot encoded vector with a matrix returns the row of the matrix corresponding the index of the "on" input unit.

![lookup](assets/lookup_matrix.png)

Instead of doing the matrix multiplication, we use the weight matrix as a lookup table. We encode the words as integers, for example "heart" is encoded as 958, "mind" as 18094. Then to get hidden layer values for "heart", you just take the 958th row of the embedding matrix. This process is called an **embedding lookup** and the number of hidden units is the **embedding dimension**.

<img src='assets/tokenize_lookup.png' width=500>
 
There is nothing magical going on here. The embedding lookup table is just a weight matrix. The embedding layer is just a hidden layer. The lookup is just a shortcut for the matrix multiplication. The lookup table is trained just like any weight matrix as well.

Embeddings aren't only used for words of course. You can use them for any model where you have a massive number of classes. A particular type of model called **Word2Vec** uses the embedding layer to find vector representations of words that contain semantic meaning.



## Word2Vec

The word2vec algorithm finds much more efficient representations by finding vectors that represent the words. These vectors also contain semantic information about the words. Words that show up in similar contexts, such as "black", "white", and "red" will have vectors near each other. There are two architectures for implementing word2vec, CBOW (Continuous Bag-Of-Words) and Skip-gram.

<img src="assets/word2vec_architectures.png" width="500">

In this implementation, we'll be using the skip-gram architecture because it performs better than CBOW. Here, we pass in a word and try to predict the words surrounding it in the text. In this way, we can train the network to learn representations for words that show up in similar contexts.

First up, importing packages.

In [1]:
import time

import numpy as np
import tensorflow as tf

import utils

Load the [text8 dataset](http://mattmahoney.net/dc/textdata.html), a file of cleaned up Wikipedia articles from Matt Mahoney. The next cell will download the data set to the `data` folder. Then you can extract it and delete the archive file to save storage space.

In [2]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
import zipfile

dataset_folder_path = 'data'
dataset_filename = 'text8.zip'
dataset_name = 'Text8 Dataset'

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile(dataset_filename):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc=dataset_name) as pbar:
        urlretrieve(
            'http://mattmahoney.net/dc/text8.zip',
            dataset_filename,
            pbar.hook)

if not isdir(dataset_folder_path):
    with zipfile.ZipFile(dataset_filename) as zip_ref:
        zip_ref.extractall(dataset_folder_path)
        
with open('data/text8') as f:
    text = f.read()

Text8 Dataset: 31.4MB [05:26, 95.9KB/s]                                                                                


## Preprocessing

Here I'm fixing up the text to make training easier. This comes from the `utils` module I wrote. The `preprocess` function coverts any punctuation into tokens, so a period is changed to ` <PERIOD> `. In this data set, there aren't any periods, but it will help in other NLP problems. I'm also removing all words that show up five or fewer times in the dataset. This will greatly reduce issues due to noise in the data and improve the quality of the vector representations. If you want to write your own functions for this stuff, go for it.

In [3]:
words = utils.preprocess(text)
print(words[:30])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst']


In [4]:
print("Total words: {}".format(len(words)))
print("Unique words: {}".format(len(set(words))))

Total words: 16680599
Unique words: 63641


And here I'm creating dictionaries to convert words to integers and backwards, integers to words. The integers are assigned in descending frequency order, so the most frequent word ("the") is given the integer 0 and the next most frequent is 1 and so on. The words are converted to integers and stored in the list `int_words`.

In [5]:
vocab_to_int, int_to_vocab = utils.create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

In [6]:
int_to_vocab[3]

'one'

## Subsampling

Words that show up often such as "the", "of", and "for" don't provide much context to the nearby words. If we discard some of them, we can remove some of the noise from our data and in return get faster training and better representations. This process is called subsampling by Mikolov. For each word $w_i$ in the training set, we'll discard it with probability given by 

$$ P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}} $$

where $t$ is a threshold parameter and $f(w_i)$ is the frequency of word $w_i$ in the total dataset.

I'm going to leave this up to you as an exercise. This is more of a programming challenge, than about deep learning specifically. But, being able to prepare your data for your network is an important skill to have. Check out my solution to see how I did it.

> **Exercise:** Implement subsampling for the words in `int_words`. That is, go through `int_words` and discard each word given the probablility $P(w_i)$ shown above. Note that $P(w_i)$ is the probability that a word is discarded. Assign the subsampled data to `train_words`.

In [7]:
## Your code here
import numpy as np
import random

t = 1e-5

p_word = np.zeros(len(int_to_vocab))
for i in int_words:
    p_word[i] += 1

p_word = 1 - np.sqrt(t / (p_word / len(int_words)))

# The final subsampled word list    
train_words = [word for word in int_words if p_word[word] > random.random()]

## Making batches

Now that our data is in good shape, we need to get it into the proper form to pass it into our network. With the skip-gram architecture, for each word in the text, we want to grab all the words in a window around that word, with size $C$. 

From [Mikolov et al.](https://arxiv.org/pdf/1301.3781.pdf): 

"Since the more distant words are usually less related to the current word than those close to it, we give less weight to the distant words by sampling less from those words in our training examples... If we choose $C = 5$, for each training word we will select randomly a number $R$ in range $< 1; C >$, and then use $R$ words from history and $R$ words from the future of the current word as correct labels."

> **Exercise:** Implement a function `get_target` that receives a list of words, an index, and a window size, then returns a list of words in the window around the index. Make sure to use the algorithm described above, where you choose a random number of words from the window.

In [47]:
def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index. '''
    
    # Your code here
    r = random.choice(range(1, window_size + 1))
    hist_idx = idx - r if idx - r > 0 else 0
    future_idx = idx + r
    
    target_words = set(words[hist_idx:idx] + words[idx + 1: future_idx + 1])
    
    return list(target_words)

Here's a function that returns batches for our network. The idea is that it grabs `batch_size` words from a words list. Then for each of those words, it gets the target words in the window. I haven't found a way to pass in a random number of target words and get it to work with the architecture, so I make one row per input-target pair. This is a generator function by the way, helps save memory.

In [48]:
def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(words)//batch_size
    
    # only full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y
    

## Building the graph

From [Chris McCormick's blog](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/), we can see the general structure of our network.
![embedding_network](./assets/skip_gram_net_arch.png)

The input words are passed in as integers. This will go into a hidden layer of linear units, then into a softmax layer. We'll use the softmax layer to make a prediction like normal.

The idea here is to train the hidden layer weight matrix to find efficient representations for our words. We can discard the softmax layer becuase we don't really care about making predictions with this network. We just want the embedding matrix so we can use it in other networks we build from the dataset.

I'm going to have you build the graph in stages now. First off, creating the `inputs` and `labels` placeholders like normal.

> **Exercise:** Assign `inputs` and `labels` using `tf.placeholder`. We're going to be passing in integers, so set the data types to `tf.int32`. The batches we're passing in will have varying sizes, so set the batch sizes to [`None`]. To make things work later, you'll need to set the second dimension of `labels` to `None` or `1`.

In [49]:
train_graph = tf.Graph()
with train_graph.as_default():
    inputs = tf.placeholder(tf.int32, [None], name='inputs')
    labels = tf.placeholder(tf.int32, [None, 1], name='labels')

## Embedding



The embedding matrix has a size of the number of words by the number of units in the hidden layer. So, if you have 10,000 words and 300 hidden units, the matrix will have size $10,000 \times 300$. Remember that we're using tokenized data for our inputs, usually as integers, where the number of tokens is the number of words in our vocabulary.


> **Exercise:** Tensorflow provides a convenient function [`tf.nn.embedding_lookup`](https://www.tensorflow.org/api_docs/python/tf/nn/embedding_lookup) that does this lookup for us. You pass in the embedding matrix and a tensor of integers, then it returns rows in the matrix corresponding to those integers. Below, set the number of embedding features you'll use (200 is a good start), create the embedding matrix variable, and use `tf.nn.embedding_lookup` to get the embedding tensors. For the embedding matrix, I suggest you initialize it with a uniform random numbers between -1 and 1 using [tf.random_uniform](https://www.tensorflow.org/api_docs/python/tf/random_uniform).

In [50]:
n_vocab = len(int_to_vocab)
n_embedding =  300 # Number of embedding features 
with train_graph.as_default():
    embedding = tf.Variable(tf.truncated_normal((n_vocab, n_embedding), stddev=0.1)) # create embedding weight matrix here
    embed = tf.nn.embedding_lookup(embedding, inputs) # use tf.nn.embedding_lookup to get the hidden layer output

## Negative sampling



For every example we give the network, we train it using the output from the softmax layer. That means for each input, we're making very small changes to millions of weights even though we only have one true example. This makes training the network very inefficient. We can approximate the loss from the softmax layer by only updating a small subset of all the weights at once. We'll update the weights for the correct label, but only a small number of incorrect labels. This is called ["negative sampling"](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf). Tensorflow has a convenient function to do this, [`tf.nn.sampled_softmax_loss`](https://www.tensorflow.org/api_docs/python/tf/nn/sampled_softmax_loss).

> **Exercise:** Below, create weights and biases for the softmax layer. Then, use [`tf.nn.sampled_softmax_loss`](https://www.tensorflow.org/api_docs/python/tf/nn/sampled_softmax_loss) to calculate the loss. Be sure to read the documentation to figure out how it works.

In [51]:
# Number of negative labels to sample
n_sampled = 100
with train_graph.as_default():
    softmax_w = tf.Variable(tf.truncated_normal((n_vocab, n_embedding), stddev=0.1)) # create softmax weight matrix here
    softmax_b = tf.Variable(tf.zeros(n_vocab)) # create softmax biases here
    
    # Calculate the loss using negative sampling
    loss = tf.nn.sampled_softmax_loss(
                weights=softmax_w,
                biases=softmax_b,
                inputs=embed,
                labels=labels,
                num_sampled=100,
                num_classes=n_vocab)
    
    cost = tf.reduce_mean(loss)
    optimizer = tf.train.AdamOptimizer().minimize(cost)

## Validation

This code is from Thushan Ganegedara's implementation. Here we're going to choose a few common words and few uncommon words. Then, we'll print out the closest words to them. It's a nice way to check that our embedding table is grouping together words with similar semantic meanings.

In [52]:
with train_graph.as_default():
    ## From Thushan Ganegedara's implementation
    valid_size = 16 # Random set of words to evaluate similarity on.
    valid_window = 100
    # pick 8 samples from (0,100) and (1000,1100) each ranges. lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples, 
                               random.sample(range(1000,1000+valid_window), valid_size//2))

    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keep_dims=True))
    normalized_embedding = embedding / norm
    valid_embedding = tf.nn.embedding_lookup(normalized_embedding, valid_dataset)
    similarity = tf.matmul(valid_embedding, tf.transpose(normalized_embedding))

In [53]:
# If the checkpoints directory doesn't exist:
!mkdir checkpoints

子目录或文件 checkpoints 已经存在。


## Training

Below is the code to train the network. Every 100 batches it reports the training loss. Every 1000 batches, it'll print out the validation words.

In [ ]:
epochs = 10
batch_size = 1000
window_size = 10

with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    iteration = 1
    loss = 0
    sess.run(tf.global_variables_initializer())

    for e in range(1, epochs+1):
        batches = get_batches(train_words, batch_size, window_size)
        start = time.time()
        for x, y in batches:
            
            feed = {inputs: x,
                    labels: np.array(y)[:, None]}
            train_loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            
            loss += train_loss
            
            if iteration % 100 == 0: 
                end = time.time()
                print("Epoch {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Avg. Training loss: {:.4f}".format(loss/100),
                      "{:.4f} sec/batch".format((end-start)/100))
                loss = 0
                start = time.time()
            
            if iteration % 1000 == 0:
                ## From Thushan Ganegedara's implementation
                # note that this is expensive (~20% slowdown if computed every 500 steps)
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = int_to_vocab[valid_examples[i]]
                    top_k = 8 # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k+1]
                    log = 'Nearest to %s:' % valid_word
                    for k in range(top_k):
                        close_word = int_to_vocab[nearest[k]]
                        log = '%s %s,' % (log, close_word)
                    print(log)
            
            iteration += 1
    save_path = saver.save(sess, "checkpoints/text8.ckpt")
    embed_mat = sess.run(normalized_embedding)

Epoch 1/10 Iteration: 100 Avg. Training loss: 8.6656 0.0648 sec/batch
Epoch 1/10 Iteration: 200 Avg. Training loss: 6.5332 0.0516 sec/batch
Epoch 1/10 Iteration: 300 Avg. Training loss: 5.2457 0.0548 sec/batch
Epoch 1/10 Iteration: 400 Avg. Training loss: 4.7898 0.0521 sec/batch
Epoch 1/10 Iteration: 500 Avg. Training loss: 4.6040 0.0527 sec/batch
Epoch 1/10 Iteration: 600 Avg. Training loss: 4.4455 0.0524 sec/batch
Epoch 1/10 Iteration: 700 Avg. Training loss: 4.3187 0.0526 sec/batch
Epoch 1/10 Iteration: 800 Avg. Training loss: 4.2296 0.0528 sec/batch
Epoch 1/10 Iteration: 900 Avg. Training loss: 4.3484 0.0520 sec/batch
Epoch 1/10 Iteration: 1000 Avg. Training loss: 4.3241 0.0518 sec/batch
Nearest to its: the, an, but, which, a, used, if, this,
Nearest to known: such, all, often, an, its, many, there, different,
Nearest to only: but, has, some, most, with, used, so, each,
Nearest to one: nine, two, seven, six, four, eight, three, d,
Nearest to between: with, age, greek, main, dead, h

Epoch 1/10 Iteration: 4600 Avg. Training loss: 4.0888 0.1663 sec/batch
Epoch 1/10 Iteration: 4700 Avg. Training loss: 4.0704 0.1590 sec/batch
Epoch 1/10 Iteration: 4800 Avg. Training loss: 4.1184 0.1598 sec/batch
Epoch 1/10 Iteration: 4900 Avg. Training loss: 4.1057 0.1581 sec/batch
Epoch 1/10 Iteration: 5000 Avg. Training loss: 4.0951 0.0525 sec/batch
Nearest to its: the, this, it, itself, which, part, continues, spread,
Nearest to known: referred, regarded, well, as, described, served, opposed, part,
Nearest to only: always, actually, thus, all, which, some, instead, but,
Nearest to one: seven, eight, four, nine, six, two, three, five,
Nearest to between: within, thus, parts, with, through, the, obligations, majority,
Nearest to he: him, his, she, himself, died, later, career, had,
Nearest to of: within, part, historical, great, from, hume, critical, and,
Nearest to most: many, outside, today, still, particularly, some, within, treatment,
Nearest to test: hole, cola, cooling, falling

Epoch 1/10 Iteration: 8900 Avg. Training loss: 4.0762 0.0527 sec/batch
Epoch 1/10 Iteration: 9000 Avg. Training loss: 4.0829 0.0517 sec/batch
Nearest to its: the, economy, rapidly, itself, it, continues, expansion, membership,
Nearest to known: referred, regarded, as, described, well, part, called, perhaps,
Nearest to only: always, actually, it, not, yet, interfaces, even, all,
Nearest to one: seven, four, two, eight, six, five, three, nine,
Nearest to between: within, thus, with, difference, obligations, parts, and, zappa,
Nearest to he: him, she, his, himself, died, returned, friend, career,
Nearest to of: the, within, habsburg, sail, historical, great, towers, and,
Nearest to most: many, particularly, still, some, among, especially, greatest, today,
Nearest to test: transmission, check, alternatively, balanced, minimum, hole, load, zappa,
Nearest to troops: army, soldiers, allied, captured, battle, occupation, defeat, allies,
Nearest to cost: speed, price, higher, weight, low, costs

Epoch 2/10 Iteration: 12800 Avg. Training loss: 3.9071 0.0528 sec/batch
Epoch 2/10 Iteration: 12900 Avg. Training loss: 3.9669 0.0521 sec/batch
Epoch 2/10 Iteration: 13000 Avg. Training loss: 4.0639 0.0519 sec/batch
Nearest to its: the, itself, expansion, it, rapidly, continues, partly, their,
Nearest to known: referred, regarded, described, remembered, perhaps, as, depicted, serves,
Nearest to only: actually, yet, it, instead, not, always, ready, however,
Nearest to one: seven, four, eight, six, five, three, two, nine,
Nearest to between: with, thus, within, difference, sierra, the, and, winds,
Nearest to he: himself, him, she, his, it, never, friend, briefly,
Nearest to of: the, lex, within, and, falkland, given, thus, whose,
Nearest to most: many, particularly, among, still, greatest, some, throughout, especially,
Nearest to test: launch, program, powered, dec, match, machine, spacecraft, mission,
Nearest to troops: forces, army, soldiers, allied, invasion, defeat, occupation, defea

Epoch 2/10 Iteration: 16800 Avg. Training loss: 4.0425 0.0535 sec/batch
Epoch 2/10 Iteration: 16900 Avg. Training loss: 4.0159 0.0524 sec/batch
Epoch 2/10 Iteration: 17000 Avg. Training loss: 4.0227 0.0533 sec/batch
Nearest to its: the, itself, it, their, expansion, pirate, roof, partly,
Nearest to known: referred, regarded, described, remembered, as, called, perhaps, seen,
Nearest to only: actually, instead, normally, not, still, similarly, a, always,
Nearest to one: seven, eight, four, six, five, nine, two, three,
Nearest to between: with, difference, thus, sierra, within, border, lie, flanders,
Nearest to he: him, his, himself, she, friend, it, retirement, wife,
Nearest to of: the, meme, honda, withdraw, descriptive, lex, eugenics, daredevil,
Nearest to most: many, some, particularly, highly, still, extremely, more, widely,
Nearest to test: program, launch, match, powered, module, launched, prototype, matches,
Nearest to troops: army, soldiers, forces, allied, armies, fleet, attack,

...

Nearest to question: questions, answer, debate, what, whether, unclear, debated, doubt,
Epoch 4/10 Iteration: 48100 Avg. Training loss: 4.0110 0.0532 sec/batch
Epoch 4/10 Iteration: 48200 Avg. Training loss: 4.0573 0.0532 sec/batch
Epoch 5/10 Iteration: 48300 Avg. Training loss: 4.0345 0.0469 sec/batch
Epoch 5/10 Iteration: 48400 Avg. Training loss: 4.0237 0.0531 sec/batch
Epoch 5/10 Iteration: 48500 Avg. Training loss: 4.0327 0.0525 sec/batch
Epoch 5/10 Iteration: 48600 Avg. Training loss: 4.0016 0.0538 sec/batch
Epoch 5/10 Iteration: 48700 Avg. Training loss: 3.9833 0.0527 sec/batch
Epoch 5/10 Iteration: 48800 Avg. Training loss: 3.9871 0.0522 sec/batch
Epoch 5/10 Iteration: 48900 Avg. Training loss: 3.9129 0.0519 sec/batch
Epoch 5/10 Iteration: 49000 Avg. Training loss: 3.9242 0.0519 sec/batch
Nearest to its: their, itself, the, it, his, stable, location, original,
Nearest to known: referred, remembered, regarded, called, described, used, understood, marketed,
Nearest to only: it, a

Epoch 5/10 Iteration: 52100 Avg. Training loss: 4.0232 0.0535 sec/batch
Epoch 5/10 Iteration: 52200 Avg. Training loss: 4.0372 0.0520 sec/batch
Epoch 5/10 Iteration: 52300 Avg. Training loss: 4.0094 0.0519 sec/batch
Epoch 5/10 Iteration: 52400 Avg. Training loss: 4.0112 0.0525 sec/batch
Epoch 5/10 Iteration: 52500 Avg. Training loss: 4.0197 0.0531 sec/batch
Epoch 5/10 Iteration: 52600 Avg. Training loss: 4.0427 0.0532 sec/batch
Epoch 5/10 Iteration: 52700 Avg. Training loss: 4.0000 0.0526 sec/batch
Epoch 5/10 Iteration: 52800 Avg. Training loss: 4.0043 0.0531 sec/batch
Epoch 5/10 Iteration: 52900 Avg. Training loss: 3.9523 0.0533 sec/batch
Epoch 5/10 Iteration: 53000 Avg. Training loss: 4.0296 0.0527 sec/batch
Nearest to its: their, itself, it, the, his, distinctive, stable, mazda,
Nearest to known: referred, remembered, regarded, described, called, such, well, used,
Nearest to only: still, normally, a, any, instead, not, enough, both,
Nearest to one: seven, four, eight, five, six, two

Epoch 5/10 Iteration: 56200 Avg. Training loss: 4.0345 0.0531 sec/batch
Epoch 5/10 Iteration: 56300 Avg. Training loss: 4.0008 0.0516 sec/batch
Epoch 5/10 Iteration: 56400 Avg. Training loss: 4.0226 0.0519 sec/batch
Epoch 5/10 Iteration: 56500 Avg. Training loss: 4.0152 0.0519 sec/batch
Epoch 5/10 Iteration: 56600 Avg. Training loss: 3.9798 0.0531 sec/batch
Epoch 5/10 Iteration: 56700 Avg. Training loss: 4.0261 0.0539 sec/batch
Epoch 5/10 Iteration: 56800 Avg. Training loss: 4.0157 0.0532 sec/batch
Epoch 5/10 Iteration: 56900 Avg. Training loss: 3.9724 0.0523 sec/batch
Epoch 5/10 Iteration: 57000 Avg. Training loss: 4.0161 0.0526 sec/batch
Nearest to its: their, the, it, itself, of, mazda, decreased, has,
Nearest to known: referred, regarded, described, called, remembered, such, defined, well,
Nearest to only: not, it, a, normally, still, that, any, both,
Nearest to one: two, four, seven, five, eight, six, nine, three,
Nearest to between: difference, both, with, distinction, relationsh

Epoch 6/10 Iteration: 60300 Avg. Training loss: 4.0333 0.0189 sec/batch
Epoch 6/10 Iteration: 60400 Avg. Training loss: 4.0330 0.0516 sec/batch
Epoch 6/10 Iteration: 60500 Avg. Training loss: 4.0059 0.0521 sec/batch
Epoch 6/10 Iteration: 60600 Avg. Training loss: 4.0379 0.0524 sec/batch
Epoch 6/10 Iteration: 60700 Avg. Training loss: 3.9747 0.0509 sec/batch
Epoch 6/10 Iteration: 60800 Avg. Training loss: 3.9841 0.0525 sec/batch
Epoch 6/10 Iteration: 60900 Avg. Training loss: 3.9774 0.0519 sec/batch
Epoch 6/10 Iteration: 61000 Avg. Training loss: 3.8736 0.0514 sec/batch
Nearest to its: their, the, it, itself, mazda, region, his, stable,
Nearest to known: referred, remembered, regarded, described, called, such, understood, as,
Nearest to only: still, not, that, a, it, the, instead, to,
Nearest to one: seven, four, eight, five, six, nine, two, three,
Nearest to between: difference, with, both, three, within, poles, across, respectively,
Nearest to he: his, she, himself, him, was, it, fath

Epoch 6/10 Iteration: 64500 Avg. Training loss: 4.0162 0.0523 sec/batch
Epoch 6/10 Iteration: 64600 Avg. Training loss: 4.0339 0.0514 sec/batch
Epoch 6/10 Iteration: 64700 Avg. Training loss: 4.0037 0.0530 sec/batch
Epoch 6/10 Iteration: 64800 Avg. Training loss: 4.0003 0.0528 sec/batch
Epoch 6/10 Iteration: 64900 Avg. Training loss: 3.9399 0.0522 sec/batch
Epoch 6/10 Iteration: 65000 Avg. Training loss: 4.0074 0.0525 sec/batch
Nearest to its: their, the, it, itself, mazda, of, his, hence,
Nearest to known: referred, regarded, described, remembered, called, such, well, marketed,
Nearest to only: instead, not, all, a, it, any, that, still,
Nearest to one: four, seven, five, six, eight, two, nine, three,
Nearest to between: difference, both, gap, with, relationship, distinction, within, split,
Nearest to he: his, she, himself, him, it, father, career, was,
Nearest to of: the, and, in, to, a, one, by, three,
Nearest to most: more, many, some, less, highly, greatest, especially, arguably,


Restore the trained network if you need to:

In [ ]:
with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    embed_mat = sess.run(embedding)

## Visualizing the word vectors

Below we'll use T-SNE to visualize how our high-dimensional word vectors cluster together. T-SNE is used to project these vectors into two dimensions while preserving local stucture. Check out [this post from Christopher Olah](http://colah.github.io/posts/2014-10-Visualizing-MNIST/) to learn more about T-SNE and other ways to visualize high-dimensional data.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:
viz_words = 500
tsne = TSNE()
embed_tsne = tsne.fit_transform(embed_mat[:viz_words, :])

In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))
for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :], color='steelblue')
    plt.annotate(int_to_vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)